<a href="https://colab.research.google.com/github/charlesprice3/Roulette/blob/main/Roulette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook acts as a simulation of a Roulette Wheel and aims to see if positive EV can be achieved through finding an edge.

TODO:

*   Implement Roulette Wheel Logic
*   Implement Roulette Memory Sliding Window
*   Implement mean reversion or ride the trend.

(Statistically speaking, each event is independently occurring, previous rolls bear no weight to future outcomes. This notebook tries to show this.)

*   Graph simulation of N playthorughs and see if we end up with positive EV



In [2]:
import random
import secrets
import numpy as np
import time
from IPython.display import clear_output  # This is the secret sauce for Notebooks

In [3]:
# Cell shows how small the offset is
memory = []
odd = 0
even = 0
n = 100000
for i in range(n):

  test = secrets.choice(range(0, 36))
  if(test % 2 == 0):
    even += 1
  else:
    odd += 1

print("Even: ", even)
print(n / 2 - even)

print("Odd: ", odd)
print(n /2 - odd)

Even:  49767
233.0
Odd:  50233
-233.0


In [4]:
# Possible Routes
# Sum the memory box, take average, use as expected value for either specific value, or choose odd or even
# Check count of each odd/even occurrence, then either
  # "Ride the trend"
  # Use mean reversion concept

# More specifically, we can count longest values in a row that are odd or even from the back, and then make a sized bet based on how large this number is

CLEANUP TODO


1.   Create Populate Function (Fills Memory)
2.   Create Roll Function (Does next roll, updates memory by one)
3.   Build Strategy Functions
4.   Build Game Logic Function





In [5]:
def populate():
  scores = []
  for i in range(10):
    scores.append(secrets.choice(range(0, 36)))
  return scores

In [6]:
def roll(scores):
  scores.pop(0)
  scores.append(secrets.choice(range(0, 36)))
  return scores

In [7]:
scores = populate()
print(scores)
scores = roll(scores)
print(scores)

[3, 31, 19, 4, 12, 10, 8, 9, 33, 21]
[31, 19, 4, 12, 10, 8, 9, 33, 21, 6]


In [8]:
# Simple Counter Function
# Based on number of odd/even occurrences, we will bet if there is a skewed amount
# Size of bet is based off of skewness
def simple_counter(scores):
  odd = 0
  even = 0
  for i in range(len(scores)):
    if(scores[i] % 2 == 0):
      even += 1
    else:
      odd += 1
  return odd, even

In [9]:
odd, even = simple_counter(scores)


In [10]:
def bet(choice, amount, scores):
  scores = roll(scores)
  if(choice == "Even"):
    if(scores[9] % 2 == 0):
      amount *= 2
    else:
      amount = 0
  elif(choice == "Odd"):
    if(scores[9] % 2 != 0):
      amount *= 2
    else:
      amount = 0
  return amount

In [ ]:
# Game logic
def play():
  cash = 1000
  i = 0
  while(cash > 1):
    print(scores)
    odd, even = simple_counter(scores)
    print("i = ", i)
    print("cash = ", cash)
    print("odd = ", odd)
    print("even = ", even)
    if(odd > 8):
      betAmount = cash / 2
      cash -= betAmount
      betAmount = bet("Even", betAmount, scores)
      cash += betAmount
    elif(even > 8):
      betAmount = cash / 2
      cash -= betAmount
      betAmount = bet("Odd", betAmount, scores)
      cash += betAmount
    else:
      roll(scores)


    i += 1
    print("i = ", i)
    print("cash = ", cash)
    print("odd = ", odd)
    print("even = ", even)
    time.sleep(.01)


    if(cash > 100000):
      return i


iter = play()
print(iter)


In [202]:
# Game logic
def play2():
  populate()
  cash = 10
  maxCash = 0
  i = 0

  # If the scoreboard is showing a skewed amount of odds or evens, we do a sized bet of half our port on the type appeared less
  # We implement a trailing stop loss, that way, we get out before our whole port is gone, however, tricky to time due to betAmount size
  # Trailing can be after one loss, or two, but given the binary nature, much of our port would already be wiped.
  while(cash > 1):
    odd, even = simple_counter(scores) # Gather Precurring Values in Scoreboard
    if(odd > 8): # Bet on Even
      betAmount = cash / 2
      cash -= betAmount
      betAmount = bet("Even", betAmount, scores)
      cash += betAmount
    elif(even > 8): # Bet on Odd
      betAmount = cash / 2
      cash -= betAmount
      betAmount = bet("Odd", betAmount, scores)
      cash += betAmount
    else: # Skip bet
      roll(scores)



    i += 1

    if(cash > maxCash):
      maxCash = cash

  return maxCash


sumBet = []
for i in range(10):
  maxCashDuring = play2()
  print(maxCashDuring)
  sumBet.append(maxCashDuring)

# We sum up all our values, it will show all our max moments, we then divide by two since we will quit after our first loss
totalThrough = sum(sumBet)
PNL = (totalThrough / 2) - 100
print("PNL: ", PNL)




56.953125
81.09146118164062
86.59522734582424
64.072265625
11.25
12.65625
15.0
5.0
33.75
5.0
PNL:  85.68416457623243


In [13]:
# Counter Function
# We iterate from the back of the scoreboard, if our scoreboard has a long consecutive line of odds or evens, then we bet based on length.
# 5 in a row, small amount